In [1]:
!pip install pytorch-forecasting pytorch-lightning pandas numpy matplotlib torch requests zipfile36


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 344.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 479.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 442.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 429.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 649.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 132.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 131.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 133.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 132.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 131.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 131.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.

In [40]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)



In [75]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

In [76]:
# Load dataset (modify file path accordingly)
train_df = pd.read_csv("data/train.csv")
val_df = pd.read_csv("data/validate.csv")
test_df = pd.read_csv("data/test.csv")

# Check for NaN values
print("Checking dataset for NaN values before preprocessing...")
print(train_df.isna().sum(), "\n")
print(val_df.isna().sum(), "\n")
print(test_df.isna().sum(), "\n")

# Fill NaNs with forward fill
train_df.fillna(method="ffill", inplace=True)
val_df.fillna(method="ffill", inplace=True)
test_df.fillna(method="ffill", inplace=True)

# Ensure target column (RUL) has no NaNs or Inf values
for df in [train_df, val_df, test_df]:
    df["RUL"].replace([np.inf, -np.inf], np.nan, inplace=True)
    df["RUL"].fillna(df["RUL"].median(), inplace=True)

print("Checking dataset for NaN values after preprocessing...")
print(train_df.isna().sum(), "\n")
print(val_df.isna().sum(), "\n")
print(test_df.isna().sum(), "\n")


Checking dataset for NaN values before preprocessing...
unit                     0
time                     0
operational_setting_1    0
operational_setting_2    0
operational_setting_3    0
sensor_1                 0
sensor_2                 0
sensor_3                 0
sensor_4                 0
sensor_5                 0
sensor_6                 0
sensor_7                 0
sensor_8                 0
sensor_9                 0
sensor_10                0
sensor_11                0
sensor_12                0
sensor_13                0
sensor_14                0
RUL                      0
dtype: int64 

unit                     0
time                     0
operational_setting_1    0
operational_setting_2    0
operational_setting_3    0
sensor_1                 0
sensor_2                 0
sensor_3                 0
sensor_4                 0
sensor_5                 0
sensor_6                 0
sensor_7                 0
sensor_8                 0
sensor_9                 0
sensor_10  

In [77]:
# Select feature columns (excluding 'time', 'unit', and target 'RUL')
input_features = [col for col in train_df.columns if col not in ["time", "unit", "RUL"]]

# Normalize input features
scaler = StandardScaler()
train_df[input_features] = scaler.fit_transform(train_df[input_features])
val_df[input_features] = scaler.transform(val_df[input_features])
test_df[input_features] = scaler.transform(test_df[input_features])


In [78]:
class TimeSeriesDataset(Dataset):
    def __init__(self, df, sequence_length=50):
        self.data = df[input_features].values
        self.targets = df["RUL"].values
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.data) - self.sequence_length

    def __getitem__(self, index):
        x = self.data[index : index + self.sequence_length]
        y = self.targets[index + self.sequence_length]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32).view(-1)

In [79]:
train_dataset = TimeSeriesDataset(train_df)
val_dataset = TimeSeriesDataset(val_df)
test_dataset = TimeSeriesDataset(test_df)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [80]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=64, num_layers=2):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        return self.fc(lstm_out[:, -1, :])  # Use last timestep's output

In [81]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = LSTMModel(input_size=len(input_features)).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Lower learning rate to improve stability

In [82]:
for inputs, targets in train_loader:
    print("Input shape:", inputs.shape)  # Should be [batch_size, sequence_length, num_features]
    print("Target shape:", targets.shape)  # Should be [batch_size, 1]
    break

Input shape: torch.Size([32, 50, 17])
Target shape: torch.Size([32, 1])


In [90]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    train_loss = 0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        targets = targets.view(-1, 1)  # Ensure correct shape

        loss = criterion(outputs, targets)

        # Skip batch if loss is NaN
        if torch.isnan(loss):
            print("NaN detected in loss! Skipping batch.")
            continue

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_train_loss:.4f}")

Epoch 1/10, Loss: 829.8437
Epoch 2/10, Loss: 829.3330
Epoch 3/10, Loss: 829.8010
Epoch 4/10, Loss: 829.4740
Epoch 5/10, Loss: 829.5112
Epoch 6/10, Loss: 828.4352
Epoch 7/10, Loss: 829.3663
Epoch 8/10, Loss: 828.7155
Epoch 9/10, Loss: 829.0584
Epoch 10/10, Loss: 828.9193


In [88]:
model.eval()
val_loss = 0

with torch.no_grad():
    for inputs, targets in val_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        outputs = model(inputs)

        targets = targets.view(-1, 1)  # Ensure correct shape

        loss = criterion(outputs, targets)
        val_loss += loss.item()

avg_val_loss = val_loss / len(val_loader)
print(f"Validation Loss: {avg_val_loss:.4f}")

Validation Loss: 816.0219


In [89]:
model_path = "lstm_model.pth"
torch.save(model.state_dict(), model_path)
print(f"✅ Model saved at {model_path}!")

✅ Model saved at lstm_model.pth!
